In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.feature
import pandas as pd
import time
import math
from ast import literal_eval
%matplotlib inline

# Patch Extraction Example  
I use blob detection to extract the dotted-area in the image

In [5]:
# data initialization
classes = ["adult_males", "subadult_males", "adult_females", "juveniles", "pups", "total"]

train_path = '/Users/YINAN/Local/Sea-lions/Data/Train/'
train_dotted_path = '/Users/YINAN/Local/Sea-lions/Data/TrainDotted/'

bad_images = [3,7,9,21,30,34,71,81,89,97,151,184,215,234,242,268,290,311,331,344,380,384,406,421,469,475,490,499,507,
              530,531,605,607,614,621,638,644,687,712,721]

file_names = [str(x) + '.jpg' for x in range(0,750) if x not in bad_images]
coordinates_df = pd.DataFrame(index=file_names, columns=classes)


In [6]:
coordinates_df = pd.read_csv("./coordinates.csv", index_col='index', converters={"total": literal_eval})

In [ ]:
# get coordinates
start = time.time()

for filename in file_names:

    # read the Train and Train Dotted images
    image_1 = cv2.imread(train_dotted_path + filename)
    image_2 = cv2.imread(train_path + filename)

    # absolute difference between Train and Train Dotted
    image_3 = cv2.absdiff(image_1,image_2)

    # mask out blackened regions from Train Dotted
    mask_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    mask_1[mask_1 < 20] = 0
    mask_1[mask_1 > 0] = 255

    mask_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
    mask_2[mask_2 < 20] = 0
    mask_2[mask_2 > 0] = 255

    image_3 = cv2.bitwise_or(image_3, image_3, mask=mask_1)
    image_3 = cv2.bitwise_or(image_3, image_3, mask=mask_2) 

    # convert to grayscale to be accepted by skimage.feature.blob_log
    image_3 = cv2.cvtColor(image_3, cv2.COLOR_BGR2GRAY)

    # detect blobs
    blobs = skimage.feature.blob_log(image_3, min_sigma=3, max_sigma=4, num_sigma=1, threshold=0.02)
    

    adult_males = []
    subadult_males = []
    pups = []
    juveniles = []
    adult_females = []
    total = []

    for blob in blobs:
        # get the coordinates for each blob
        y, x, s = blob
        # get the color of the pixel from Train Dotted in the center of the blob
        g,b,r = image_1[int(y)][int(x)][:]

        # decision tree to pick the class of the blob by looking at the color in Train Dotted
        if r > 200 and g < 50 and b < 50: # RED
            adult_males.append((int(x),int(y)))        
        elif r > 200 and g > 200 and b < 50: # MAGENTA
            subadult_males.append((int(x),int(y)))         
        elif r < 100 and g < 100 and 150 < b < 200: # GREEN
            pups.append((int(x),int(y)))
        elif r < 100 and  100 < g and b < 100: # BLUE
            juveniles.append((int(x),int(y))) 
        elif r < 150 and g < 50 and b < 100:  # BROWN
            adult_females.append((int(x),int(y)))
            
        total.append((int(x), int(y)))

    coordinates_df["adult_males"][filename] = adult_males
    coordinates_df["subadult_males"][filename] = subadult_males
    coordinates_df["adult_females"][filename] = adult_females
    coordinates_df["juveniles"][filename] = juveniles
    coordinates_df["pups"][filename] = pups
    coordinates_df["total"][filename] = total
    
    print("\r%s completes..."%filename, end='')
print("\ncompletes!!!")
print("total time is {:.2f} minutes".format((time.time() - start) / 60))


In [22]:
def extract_patches_with_sealions(coordinates_df):
    patches = []
    for filename in coordinates_df.index:
        image = cv2.imread(train_path + filename)
        for coordinates in coordinates_df.loc[filename].total:
            thumb = image[coordinates[1]-48:coordinates[1]+48,coordinates[0]-48:coordinates[0]+48,:]
            if np.shape(thumb) == (96, 96, 3):
                patches.append(cv2.cvtColor(thumb, cv2.COLOR_BGR2RGB))
    return patches

In [23]:
patches = extract_patches(coordinates_df.sample(30))

In [17]:
def check_sealions(patches):
    sealions = []
    for patch in patches:
        R = np.average(patch[:,:,0])
        G = np.average(patch[:,:,1])
        B = np.average(patch[:,:,2])
        if B >= R and B >= G:
            continue
        sealions.append(patch)
    return sealions

In [24]:
sealions = check_sealions(patches)

In [25]:
len(patches)

3363

In [26]:
len(sealions)

3190

In [28]:
3190/3363*100

94.85578352661315

In [ ]:
# extract 96 x 96 patches
x = []
y = []
for filename in file_names:    
    image = cv2.imread("./data/train/" + filename)
    for lion_class in classes:
        for coordinates in coordinates_df[lion_class][filename]:
            thumb = image[coordinates[1]-48:coordinates[1]+48,coordinates[0]-48:coordinates[0]+48,:]
            if np.shape(thumb) == (96, 96, 3):
                x.append(cv2.cvtColor(thumb, cv2.COLOR_BGR2RGB))
                y.append(lion_class)
x = np.array(x)
y = np.array(y)

In [ ]:
# plot examples
for lion_class in classes:
    f, ax = plt.subplots(1,10,figsize=(24,3))
    f.suptitle(lion_class)
    axes = ax.flatten()
    j = 0
    for a in axes:
        a.set_xticks([])
        a.set_yticks([])
        for i in range(j,len(x)):
            if y[i] == lion_class:
                j = i+1
                a.imshow(x[i])
                break

In [ ]:
def calculate_error(n):
    error = 0
    for i in range(n):
        if i in bad_images: continue
        standard = stat_df.iloc[i][1:]
        calculate = list(map(lambda x: len(x), list(coordinates_df.loc[str(i)+'.jpg'])))[:-1]
        e = sum(abs(standard - calculate))
        if e > 10:
            print(i)
        error += e
    return error 

In [ ]:
stat_df = pd.read_csv('/Users/YINAN/Local/Sea-lions/Data/train.csv')

In [ ]:
calculate_error(len(file_names))

In [ ]:
coordinates_df.iloc[590]

In [ ]:
coordinates_df.loc['0.jpg']

# Sliding window to extract patches

In [69]:
test_image_name = '0.jpg'
test = cv2.imread(train_path + test_image_name)
test.shape

(3744, 5616, 3)

In [ ]:
def extract_patches_with_sealions(coordinates_df):
    patches = []
    for filename in coordinates_df.index:
        image = cv2.imread(train_path + filename)
        for coordinates in coordinates_df.loc[filename].total:
            thumb = image[coordinates[1]-48:coordinates[1]+48,coordinates[0]-48:coordinates[0]+48,:]
            if np.shape(thumb) == (96, 96, 3):
                patches.append(cv2.cvtColor(thumb, cv2.COLOR_BGR2RGB))
    return patches



def extract_sealions_without_sealions(coordinates_df):
    patches = []
    for filename in coordinates_df.index:
        sealion_coordinates_list = coordinates_df.loc[filename].total
        image = cv2.imread(train_path + filename)
        for row in range(image.shape[0]//96):
            for col in range(image.shape[1]//96):
                center = (row*96+48, col*96+48)
                flag = True
                for thumb in sealion_coordinates_list:
                    if math.sqrt((center[0] - thumb[1])**2 + (center[1] - thumb[0])**2) < math.sqrt(2)*96:
                        flag = False
                        break
                if flag:
                    patch_rgb = cv2.cvtColor(image[row*96:row*96+96, col*96:col*96+96], cv2.COLOR_BGR2RGB)
                    patches.append(patch_rgb)
    return patches 

In [70]:
patches = []
for row in range(0, test.shape[0]//96):
    for col in range(0, test.shape[1]//96):    
        center = (row*96+48, col*96+48)
        flag = True
        for thumb in coordinates_df.loc['0.jpg'].total:
            if math.sqrt((center[0] - thumb[1])**2 + (center[1] - thumb[0])**2) < math.sqrt(2)*96:
                flag = False
                break
        if flag:
            patch_rgb = cv2.cvtColor(test[row*96:row*96+96, col*96:col*96+96], cv2.COLOR_BGR2RGB)
            patches.append(patch_rgb)
                

In [71]:
len(patches)

1014

In [32]:
len(coordinates_df.loc['0.jpg'].total)

946

In [36]:
res = check_sealions(patches)

## TO DO
* using logistic regression to classify images
* generate pos and neg patches
* train the classifier 
* test

In [67]:
### extract pos patches
pos_df = pd.DataFrame(columns=["R", "G", "B"])
patches = extract_patches(coordinates_df=coordinates_df.head(1))

for i in range(len(patches)):
    patch = patches[i]
    r = np.average(patch[:,:,0])
    g = np.average(patch[:,:,1])
    b = np.average(patch[:,:,2])
    pos_df = pos_df.append({'R':r, "G":g, "B":b}, ignore_index=True)

pos_df['class'] = 1

In [76]:
### extract neg patches
neg_df = pd.DataFrame(columns=["R", "G", "B"])

for i in range(len(patches)):
    patch = patches[i]
    r = np.average(patch[:,:,0])
    g = np.average(patch[:,:,1])
    b = np.average(patch[:,:,2])
    neg_df = neg_df.append({'R':r, "G":g, "B":b}, ignore_index=True)

neg_df['class'] = 0

In [80]:
total = pos_df.append(neg_df)

In [99]:
### build logistic regression 
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import recall_score

target = 'class'
variables = total.columns[total.columns != target]

x_train, x_test,y_train, y_test = \
model_selection.train_test_split(total[["B", "G", 'R']], total[target], test_size = 0.33, random_state=11)

logistic = linear_model.LogisticRegression(C=10e10, random_state=1234)
logistic.fit(x_train, y_train)

y_pred = logistic.predict(x_test)

recall_score(y_test, y_pred)

0.93059936908517349

In [114]:
def extract_patches_with_sealions(coordinates_df):
    patches = []
    count = 0
    for filename in coordinates_df.index:
        count += 1
        image = cv2.imread(train_path + filename)
        for coordinates in coordinates_df.loc[filename].total:
            thumb = image[coordinates[1]-48:coordinates[1]+48,coordinates[0]-48:coordinates[0]+48,:]
            if np.shape(thumb) == (96, 96, 3):
                patches.append(cv2.cvtColor(thumb, cv2.COLOR_BGR2RGB))
        print("\r%d file completes, with total %d"%(count, len(coordinates_df)), end='')
    return patches



def extract_patches_without_sealions(coordinates_df):
    patches = []
    count = 0
    for filename in coordinates_df.index:
        sealion_coordinates_list = coordinates_df.loc[filename].total
        image = cv2.imread(train_path + filename)
        count += 1
        for row in range(image.shape[0]//96):
            for col in range(image.shape[1]//96):
                center = (row*96+48, col*96+48)
                flag = True
                for thumb in sealion_coordinates_list:
                    if math.sqrt((center[0] - thumb[1])**2 + (center[1] - thumb[0])**2) < math.sqrt(2)*96:
                        flag = False
                        break
                if flag:
                    patch_rgb = cv2.cvtColor(image[row*96:row*96+96, col*96:col*96+96], cv2.COLOR_BGR2RGB)
                    patches.append(patch_rgb)
        print("\r%d file completes, with total %d"%(count, len(coordinates_df)), end='')
    return patches 

In [115]:
### extract pos patches
pos_df = pd.DataFrame(columns=["R", "G", "B"])
patches = extract_patches_with_sealions(coordinates_df=coordinates_df.sample(300))

for i in range(len(patches)):
    patch = patches[i]
    r = np.average(patch[:,:,0])
    g = np.average(patch[:,:,1])
    b = np.average(patch[:,:,2])
    pos_df = pos_df.append({'R':r, "G":g, "B":b}, ignore_index=True)

pos_df['class'] = 1

300 file completes, with total 300

In [153]:
### extract neg patches
neg_df = pd.DataFrame(columns=["R", "G", "B"])
patches = extract_patches_without_sealions(coordinates_df=coordinates_df.sample(15))

for i in range(len(patches)):
    patch = patches[i]
    r = np.average(patch[:,:,0])
    g = np.average(patch[:,:,1])
    b = np.average(patch[:,:,2])
    neg_df = neg_df.append({'R':r, "G":g, "B":b}, ignore_index=True)

neg_df['class'] = 0

15 file completes, with total 15

In [154]:
len(neg_df)

28819

In [155]:
len(pos_df)

26764

In [157]:
total = pos_df.append(neg_df)

In [188]:
### build logistic regression 
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

target = 'class'
variables = total.columns[total.columns != target]

x_train, x_test,y_train, y_test = \
model_selection.train_test_split(total[["B", "G", 'R']], total[target], test_size = 0.33, random_state=11)

logistic = linear_model.LogisticRegression(C=1, random_state=1234)
logistic.fit(x_train, y_train)

y_pred = logistic.predict(x_test)

print("recall score is {}".format(recall_score(y_test, y_pred)))
print("precision score is {}".format(precision_score(y_test, y_pred)))

recall score is 0.8012102196324519
precision score is 0.7785278745644599


In [192]:
y_pred[:10]

array([1, 0, 0, 1, 1, 1, 0, 1, 0, 1])

In [194]:
logistic.predict_proba(x_test)[:10]

array([[ 0.2581087 ,  0.7418913 ],
       [ 0.63594455,  0.36405545],
       [ 0.75313516,  0.24686484],
       [ 0.24557914,  0.75442086],
       [ 0.22792968,  0.77207032],
       [ 0.49220043,  0.50779957],
       [ 0.94375765,  0.05624235],
       [ 0.13003474,  0.86996526],
       [ 0.54873745,  0.45126255],
       [ 0.19137663,  0.80862337]])

In [199]:
def ClassifyWithThreshold(probabilities, threshold):
    return [+1 if x>=threshold else 0 for x in probabilities]

In [219]:
y_pred_new = ClassifyWithThreshold(logistic.predict_proba(x_test)[:,1], 0.05)

In [220]:
recall_score(y_test, y_pred_new)

0.97725235320484083

In [221]:
precision_score(y_test, y_pred_new)

0.50789121192708642

In [212]:
logistic.predict_proba(x_test)

array([[ 0.2581087 ,  0.7418913 ],
       [ 0.63594455,  0.36405545],
       [ 0.75313516,  0.24686484],
       ..., 
       [ 0.6111643 ,  0.3888357 ],
       [ 0.70157573,  0.29842427],
       [ 0.94388032,  0.05611968]])